In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'cairns_gtfs.zip'
path = '/home/araichev/gtfs_explorer/data/raw/san_jose_gtfs_20161229.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


In [16]:
print(dt.datetime.now())
report = feed.validate()
print(dt.datetime.now())
report

2017-04-14 14:10:57.583172
2017-04-14 14:10:58.752968


,table,error,error_row_indices
0,trips,service_id undefined,"[7840, 7841, 7842, 7843, 7844, 7845, 7846, 784..."
1,trips,block_id unduplicated,"[421, 422, 1749, 2423, 2594, 3091, 3266, 3598,..."


In [15]:
f = feed.calendar_dates
f
feed.calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,Weekdays,1,1,1,1,1,0,0,20170102,20170409
1,Saturday,0,0,0,0,0,1,0,20170102,20170409
2,Sunday,0,0,0,0,0,0,1,20170102,20170409


In [12]:
pd.isnull(f['transfers'].iat[2])


True

In [ ]:
report = feed.validate()

In [ ]:
report

In [ ]:
f = feed.stop_times.copy()
i = 10
f.ix[9:11]

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()